In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 텍스트 데이터 카테고리 분류 모델 만들기
- Vectorizer + MultinomialNB

```
s1 => 우리 나라는 한국 입니다.
s2 => 한국은 살기 좋은 나라 입니다.

   우리 나라 한국 살기 좋은 
s1   1    1    1    0    0  
s2   0    1    1    1    2 
```

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
article_df = pd.read_csv("./data/articles_1200.csv")
article_df = article_df.dropna()
len(article_df)

1200

In [4]:
article_df.head()

,title,link,category,content
0,합의문 휴짓조각…“불신임”까지 터져나온 나경원,https://news.naver.com/main/read.nhn?mode=LSD&...,100,협상 과정 전략 없이 갈팡질팡나경원 자유한국당 원내대표가 24일 오후 국회에서 열린...
1,"합의문 쓰고도 무산, '파국' 치닿는 국회…정상화 언제쯤?",https://news.naver.com/main/read.nhn?mode=LSD&...,100,[앵커]지금 국회에는 서복현 기자가 남아 있습니다. 연결해서 분위기 좀 들어보겠습니...
2,“삭발까지 했는데” 의총 반란…합의문 2시간 만에 휴지조각,https://news.naver.com/main/read.nhn?mode=LSD&...,100,"ㆍ한국당, 국회 정상화 합의안 추인 거부 ‘자중지란’ㆍ의원 20여명 “얻은 게 뭐냐..."
3,"나경원 리더십 타격, 투톱 황교안도 ‘불똥’",https://news.naver.com/main/read.nhn?mode=LSD&...,100,ㆍ재위임 불구 대여 입지 축소ㆍ당내 ‘박영선 조기사퇴’ 회자ㆍ강경 일변도 황 대표 ...
4,"與, '대정부 투쟁' 민노총과 거리두기 고심",https://news.naver.com/main/read.nhn?mode=LSD&...,100,"與, 대정부 투쟁 민노총과 거리두기 고심[앵커] 민주노총이 김명환 위원장의 구속을 ..."


In [5]:
# vectorizer : 글자를 숫자로 바꾼다.
# Tfidf: Term Frequency - Inverse Document Frequency(특정 row의 단어 / 전체 row에서의 단어) = 특정 단어가 문서에서 얼마나 중요한지 파악하기 위함
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(article_df.content)
x.shape, article_df.category.shape

# ((1200, 100169), (1200,)) -> 1200개 data에 100169개 feature vetor 만들어짐

((1200, 100169), (1200,))

In [6]:
print(vectorizer.get_feature_names()[10000:10020])

['가졌던', '가졌습니다', '가졌어야', '가졌으며', '가졌을', '가졌지만', '가족', '가족과', '가족끼리', '가족단위', '가족들과', '가족들에게', '가족들은', '가족들을', '가족들의', '가족들이', '가족력이', '가족명의', '가족복지', '가족분들은']


In [7]:
len(vectorizer.vocabulary_), list(vectorizer.vocabulary_.items())[:3]

(100169, [('협상', 97560), ('과정', 16752), ('전략', 74790)])

In [8]:
train_x, test_x, train_y, test_y = train_test_split(x, article_df.category, test_size=0.2, random_state=1)

In [9]:
# 모델 학습
model = MultinomialNB().fit(train_x, train_y)

In [10]:
pred_y = model.predict(test_x)

In [11]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

         100       0.82      0.95      0.88        43
         101       0.83      0.67      0.74        43
         102       0.70      0.82      0.76        40
         103       0.77      0.53      0.63        32
         104       0.90      0.85      0.88        41
         105       0.74      0.85      0.80        41

    accuracy                           0.79       240
   macro avg       0.79      0.78      0.78       240
weighted avg       0.80      0.79      0.79       240



#### Pipeline 사용

In [19]:
# article_df.content로 문장 그대로를 넣어준다.

train_x, test_x, train_y, test_y = train_test_split(
    article_df.content, article_df.category, test_size=0.2, random_state=1)
len(train_x), len(test_x), len(train_y), len(test_y)

(960, 240, 960, 240)

In [20]:
train_x.head()

1184    한국형 원자로 핵심기술이 해외로 유출됐다는 의혹이 제기됐다. 사진은 한국형 원자로 ...
771     23일 싱가포르 컴포트 델그로 본사에서 현대차그룹 정의선 수석부회장과 양반셍 컴포트...
329     산업硏 하반기 경제·산업 전망          글로벌 경기 둔화와 미·중 무역분쟁 ...
587     <앵커>부산시와 경남도의 중요한 세금 수입인 취득세가 크게 줄어들고 있습니다. 좀처...
746     국내 26만여 노인 환자 대상 치매 발병률 조사정보영 교수 연구팀 "항응고치료 등 ...
Name: content, dtype: object

In [21]:
# pipeline 순서대로 실행
# 1. vetorizer 진행
# 2. 학습

model = Pipeline([
    ('vect', TfidfVectorizer()), 
    ('clf', MultinomialNB(alpha=0.001)),
])

In [22]:
model = model.fit(train_x, train_y)

In [23]:
pred_y = model.predict(test_x)

In [26]:
pred_y

array([102, 103, 105, 100, 105, 101, 104, 101, 102, 103, 103, 100, 101,
       100, 104, 105, 105, 105, 102, 105, 105, 105, 105, 103, 101, 104,
       102, 100, 104, 102, 105, 105, 105, 103, 100, 102, 102, 105, 104,
       100, 102, 102, 100, 100, 103, 101, 103, 103, 103, 101, 103, 101,
       105, 101, 102, 104, 102, 102, 102, 105, 100, 101, 100, 103, 100,
       101, 103, 104, 103, 105, 101, 104, 102, 105, 100, 105, 100, 101,
       104, 100, 105, 102, 100, 102, 102, 105, 103, 104, 101, 102, 105,
       102, 104, 103, 100, 105, 100, 104, 105, 102, 104, 105, 103, 101,
       102, 100, 102, 102, 100, 105, 104, 103, 104, 100, 105, 100, 105,
       101, 102, 101, 100, 100, 100, 101, 104, 100, 101, 101, 101, 104,
       104, 102, 101, 100, 102, 104, 100, 103, 102, 101, 103, 103, 105,
       105, 100, 104, 104, 100, 101, 104, 103, 102, 103, 102, 104, 103,
       100, 102, 102, 103, 103, 100, 105, 102, 100, 104, 102, 101, 100,
       105, 104, 103, 101, 104, 105, 101, 103, 103, 100, 102, 10

In [24]:
confusion_matrix(test_y, pred_y)

array([[40,  0,  2,  0,  0,  1],
       [ 0, 32,  6,  1,  0,  4],
       [ 3,  2, 33,  2,  0,  0],
       [ 1,  2,  1, 25,  0,  3],
       [ 0,  0,  1,  4, 34,  2],
       [ 0,  1,  0,  5,  1, 34]], dtype=int64)

In [25]:
# 데이터가 언밸런스하지 않기 때문에 accurary를 사용하면 된다.
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

         100       0.91      0.93      0.92        43
         101       0.86      0.74      0.80        43
         102       0.77      0.82      0.80        40
         103       0.68      0.78      0.72        32
         104       0.97      0.83      0.89        41
         105       0.77      0.83      0.80        41

    accuracy                           0.82       240
   macro avg       0.83      0.82      0.82       240
weighted avg       0.83      0.82      0.83       240



In [ ]:
# binary일 때 f1 score만 확인 가능

# from sklearn.metrics import f1_score
# f1_score(test_y, pred_y)

In [ ]:
# 예측

In [27]:
classification_dict = {
    100:"정치",
    101:"경제",
    102:"사회",
    103:"생활/문화",
    104:"세계",
    105:"IT/과학",
}

In [28]:
contents = [
    "네이버와 카카오 드론 기술 발전에 주력",
    "요즘에 환율 주가 예측 불허",
    "트럼프 미국 대통령 중국과 무역협상 타결",
]

datas = {
    "content": contents,
    "category_code": model.predict(contents),
}

df = pd.DataFrame(datas)
df["category"] = df["category_code"].apply(lambda data: classification_dict[data])
df["proba"] = df["content"].apply(lambda data: round(max(model.predict_proba([data])[0]), 2))
df

,content,category_code,category,proba
0,네이버와 카카오 드론 기술 발전에 주력,105,IT/과학,0.96
1,요즘에 환율 주가 예측 불허,101,경제,0.98
2,트럼프 미국 대통령 중국과 무역협상 타결,104,세계,0.75


In [29]:
data = '요즘에 환율 주가 예측 불허'
tuple(zip(classification_dict.values(), np.round(model.predict_proba([data])[0], 2)))

(('정치', 0.0),
 ('경제', 0.98),
 ('사회', 0.0),
 ('생활/문화', 0.0),
 ('세계', 0.0),
 ('IT/과학', 0.01))

In [30]:
data = "트럼프 미국 대통령 중국과 무역협상 타결"
tuple(zip(classification_dict.values(), np.round(model.predict_proba([data])[0], 2)))

(('정치', 0.01),
 ('경제', 0.23),
 ('사회', 0.01),
 ('생활/문화', 0.0),
 ('세계', 0.75),
 ('IT/과학', 0.0))